# OpenAPI Tool Adapter

    - Goal: Parse OpenAPI snippet
- Goal: Construct mock tool signature and response

    **Pass criteria:** Parsed endpoints mapped to deterministic mock call.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/16_openapi_tool')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
openapi_yaml = """
openapi: 3.0.0
info:
  title: Sample Weather
  version: "1.0"
paths:
  /weather:
    get:
      summary: Get weather by city
      parameters:
        - name: city
          in: query
          schema:
            type: string
      responses:
        '200':
          description: Successful response
"""
try:
    import yaml  # type: ignore
    spec = yaml.safe_load(openapi_yaml)
except Exception:
    spec = {
        "openapi": "3.0.0",
        "info": {"title": "Sample Weather", "version": "1.0"},
        "paths": {
            "/weather": {
                "get": {
                    "summary": "Get weather by city",
                    "parameters": [
                        {"name": "city", "in": "query", "schema": {"type": "string"}}
                    ],
                    "responses": {"200": {"description": "Successful response"}}
                }
            }
        }
    }
endpoint = spec["paths"]["/weather"]["get"]
tool_signature = {
    "name": "weather_lookup",
    "description": endpoint["summary"],
    "args": [param["name"] for param in endpoint.get("parameters", [])]
}
mock_result = {"city": "Seattle", "temperature_c": 12, "source": "openapi-mock"}
output = {"spec": spec["info"], "tool": tool_signature, "call_result": mock_result}
(RUN / "openapi.json").write_text(json.dumps(output, indent=2))
output

In [ ]:
meta = write_meta(ok=True, lab="16_openapi_tool", args=len(tool_signature['args']))
meta